In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F
from sklearn.utils import class_weight
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from sklearn.metrics import confusion_matrix

In [2]:
def load_data(datafile, flare_label, series_len, start_feature, n_features, mask_value):
    df=pd.read_csv(datafile)
    df_values=df.values
    X=[]
    y=[]
    tmp=[]
    for k in range (start_feature, start_feature+n_features):
        tmp.append(mask_value)
    for idx in range(0, len(df_values)):
        each_series_data=[]
        row=df_values[idx]
        if flare_label == 'M5' and row[1][0]=='M' and float(row[1][1:]) >=5.0:
            label='X'
        else:
            label=row[1][0]
        if flare_label=='M' and label=='X':
            label='M'
        if flare_label=='C' and (label=='X' or label=='M'):
            label='C'
        if flare_label=='B' and (label=='X' or label=='M' or label=='C'):
            label='B'
        if flare_label=='M5' and (label=='M' or label=='C' or label=='B'):
            label='N'
        if flare_label=='M' and (label=='B' or label=='C'):
            label='N'
        if flare_label=='C' and label=='B':
            label='N'
        has_zero_record=False
        #if at least one of the 25 physical feature values is missing, then discard it
        if flare_label=='C':
            if float(row[5])==0.0:
                has_zero_record=True
            if float(row[7])==0.0:
                has_zero_record=True
            for k in range(9, 13):
                if float(row[k])==0.0:
                    has_zero_record=True
                    break
            for k in range(14, 16):
                if float(row[k])==0.0:
                    has_zero_record=True
                    break
            for k in range(18, 21):
                if float(row[k])==0.0:
                    has_zero_record=True
                    break
            if float(row[22])==0.0:
                has_zero_record=True
            for k in range(24, 33):
                if float(row[k])==0.0:
                    has_zero_record=True
                    break
            for k in range(38, 42):
                if float(row[k])==0.0:
                    has_zero_record=True
                    break
            #check only for C flares prediction
            
            if has_zero_record is False:
                cur_noaa_num=int(row[3])
                each_series_data.append(row[start_feature:start_feature+n_features].tolist())
                itr_idx=idx-1
                while itr_idx>=0 and len(each_series_data)<series_len:
                    prev_row=df_values[itr_idx]
                    prev_noaa_num=int(prev_row[3])
                    if prev_noaa_num!=cur_noaa_num:
                        break
                    has_zero_record_tmp=False
                    if flare_label=='C':
                        if float(row[5])==0.0:
                            has_zero_record_tmp=True
                        if float(row[7])==0.0:
                            has_zero_record_tmp=True
                        for k in range(9, 13):
                            if float(row[k])==0.0:
                                has_zero_record_tmp=True
                                break
                        for k in range(14, 16):
                            if float(row[k])==0.0:
                                has_zero_record_tmp=True
                                break
                        for k in range(18, 21):
                            if float(row[k])==0.0:
                                has_zero_record_tmp=True
                                break
                        if float(row[22])==0.0:
                                has_zero_record_tmp=True
                        for k in range(24, 33):
                            if float(row[k])==0.0:
                                has_zero_record_tmp=True
                                break
                        for k in range(38, 42):
                            if float(row[k])==0.0:
                                has_zero_record_tmp=True
                                break
                                
                                #tested only for C flares
                    if len(each_series_data)<series_len and has_zero_record_tmp is True:
                        each_series_data.insert(0, tmp)
                    
                    if len(each_series_data)<series_len and has_zero_record_tmp is False:
                        each_series_data.insert(0, prev_row[start_feature:start_feature+n_features].tolist())
                    itr_idx-=1
                while len(each_series_data)>0 and len(each_series_data)<series_len:
                    each_series_data.insert(0, tmp)
                
                if len(each_series_data)>0:
                    X.append(np.array(each_series_data).reshape(series_len, n_features).tolist())
                    y.append(label)
    X_arr=np.array(X)
    y_arr=np.array(y)
    print(X_arr.shape)
    return X_arr, y_arr


                        
                

In [3]:
flare_label='C'
filepath='./'+flare_label+'/'
n_features=14
num_of_fold=10
start_feature=5
mask_value=0
series_len=10
epochs=7
batch_size=256
nclass=2
thlistsize=201
thlist=np.linspace(0, 1, thlistsize)

X_train_data, y_train_data = load_data(datafile=filepath+'normalized_training.csv', 
                                       flare_label=flare_label,
                                       series_len=series_len,
                                       start_feature=start_feature,
                                       n_features=n_features,
                                       mask_value=mask_value)

X_valid_data, y_valid_data=load_data(datafile=filepath+'normalized_validation.csv', 
                                     flare_label=flare_label,
                                     series_len=series_len,
                                     start_feature=start_feature,
                                     n_features=n_features,
                                     mask_value=mask_value)

X_test_data, y_test_data=load_data(datafile=filepath+'normalized_testing.csv',
                                   flare_label=flare_label,
                                   series_len=series_len,
                                   start_feature=start_feature,
                                   n_features=n_features,
                                   mask_value=mask_value)


(84577, 10, 14)
(26473, 10, 14)
(44689, 10, 14)


In [4]:
def data_transform(data):
    encoder=LabelEncoder()
    encoder.fit(data)
    encoded_Y=encoder.transform(data)
    print(encoded_Y)
    converteddata=F.one_hot(torch.Tensor(encoded_Y).long())
    return converteddata.float()
    

In [5]:
def data_transform_ce(data):
    encoder=LabelEncoder()
    encoder.fit(data)
    encoded_Y=encoder.transform(data)
    encoded_Y=np.ones(encoded_Y.shape)-encoded_Y
    return encoded_Y

In [6]:
class RNNModel(nn.Module):
    
    def __init__(self, rnn_layer, vocab_size, **kwards):
        super(RNNModel, self).__init__(**kwards)
        self.rnn=rnn_layer
        self.vocab_size=vocab_size
        self.num_hiddens=self.rnn.hidden_size
        
        if not self.rnn.bidirectional:
            self.num_directions=1
            self.drop=nn.Dropout(0.5)
            self.linear=nn.Linear(self.num_hiddens, 1)
            
            self.sigm=nn.Sigmoid()
        else:
            self.num_directions=2
            self.linear=nn.Linear(self.num_hiddens*2, self.vocab_size)
            
    def forward(self, inputs, state):
        
        inputs=inputs.permute(1, 0, 2)
        Y, state=self.rnn(inputs, state)
        
        h_t=Y[-1, :, :]
        
        model_output=self.linear(h_t)
        return state, model_output
    
    def begin_state(self, batch_size=1):
            #'nn.LSTM takes a tuple of hidden states'
            return (torch.zeros((self.num_directions*self.rnn.num_layers, batch_size, self.num_hiddens)), 
                    torch.zeros((self.num_directions*self.rnn.num_layers, batch_size, self.num_hiddens)))

In [7]:
def train_epoch_ch8(model, train_iter, train_exp,  updater,  p,loss,  #@save
                    use_random_iter,epoch):
    """Train a model within one epoch (defined in Chapter 8)."""
    state= None
    
    count=0
    output_all=torch.tensor([])
    Y_all=torch.tensor([])
    for X, Y in train_iter:
        if count!=train_exp:
            
            if state is None or use_random_iter:
                # Initialize `state` when either it is the first iteration or
              
                state = model.begin_state(batch_size=8457)
            else:
                if isinstance(model, nn.Module) and not isinstance(state, tuple):
                    
                    state.detach_()
                else:
                    # `state` is a tuple of tensors for `nn.LSTM` and
                   
                    for s in state:
                        s.detach_()
          
            
            state, output = model(X, state)
            output=torch.squeeze(output)
            if epoch==9:
                with torch.no_grad():
                    output_all=torch.cat((output, output_all), dim=0)
                    Y_all=torch.cat((Y, Y_all), dim=0)
           
            l=loss(output, Y )#+torch.tensor(1.0/8457)*torch.tensor(0.0001)*torch.sum(output**2)
           
            if isinstance(updater, torch.optim.Optimizer):
                updater.zero_grad()
                l.backward()
                #grad_clipping(model, 1)
                updater.step()
        count+=1
           
    if epoch==9:
        with torch.no_grad():
            cmat=confusion_matrix(Y_all, np.array([output_all[i]>(0.5) for i in range (output_all.shape[0])]))
            print(sum(Y_all))
            print(Y_all[0:20])
            print(output_all[0:20])
            print('cmat_train: ')
            print(cmat)
    return l

In [8]:
def train_ch8(model, train_iter, train_exp, class_weights, lr, num_epochs, 
              use_random_iter=False):
    """Train a model (defined in Chapter 8)."""

    loss=nn.BCEWithLogitsLoss(pos_weight=class_weights)
    # Initialize
    optim=torch.optim.Adam(model.parameters(), lr , betas = (0.9, 0.999))
    # Train and predict
    for epoch in range(num_epochs):
        l = train_epoch_ch8(
            model, train_iter,train_exp,  optim,class_weights, loss,use_random_iter, epoch)
        print(f'loss {l:.4f}')

In [9]:
lr=0.001
tensor_x = torch.Tensor(X_train_data)
tensor_y = torch.Tensor(data_transform_ce(y_train_data))
train_dataset=TensorDataset(tensor_x, tensor_y)
train_dataloader=DataLoader(train_dataset, batch_size=8457, shuffle=True, drop_last=True)
m=16
num_epochs=10

In [10]:
class_weights_all={0:[], 1:[]}

In [11]:
for X, Y in train_dataloader:
    class_weights=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(Y) , y=np.array(Y))
    class_weights_all[0].append(class_weights[0])
    class_weights_all[1].append(class_weights[1])

In [12]:
tensor_x_ts=torch.Tensor(X_test_data)
tensor_y_ts=torch.Tensor(data_transform_ce(y_test_data))
test_dataset=TensorDataset(tensor_x_ts, tensor_y_ts)
test_dataloader=DataLoader(test_dataset, batch_size=4468,  shuffle=True, drop_last=True)

In [13]:


max_recall=0
max_precision=0
max_acc=0
max_tss=0
max_far=0
max_hss=0
recall_list=[]
precision_list=[]
acc_list=[]
hss_list=[]
tss_list=[]
far_list=[]

fraction_of_positives_list=[]
mean_predicted_value_list=[]
fpr_list=[]
tpr_list=[]
num_of_folds=10


In [14]:
for train_itr in range(num_of_folds):
        print('--------- '+str(train_itr) + ' iteration-------------')
        
        test_itr=train_itr
        classweights0=(np.sum(class_weights_all[0])-class_weights_all[0][train_itr])/num_of_folds
        classweights1=(np.sum(class_weights_all[1])-class_weights_all[1][train_itr])/num_of_folds
        classweights=classweights1/ classweights0
        classweights=torch.Tensor(np.array(classweights))
        print(classweights)
        lstm_layer=nn.LSTM(n_features, m,2 )
        model=RNNModel(lstm_layer, m)
        
        train_ch8(model, train_dataloader, train_itr, classweights, lr, num_epochs)
        
        y_ts_all=torch.tensor([])
        output_ts_all=torch.tensor([])
        
        count=0
        for i in test_dataloader:
            if count!=test_itr:
                x, y=i
                output=model(x, model.begin_state(x.shape[0]))
                y_ts_all=torch.cat((y, y_ts_all))
                output_ts_all=torch.cat((torch.squeeze(output[1]),output_ts_all), dim=0)
            count+=1
        loss=nn.BCEWithLogitsLoss(pos_weight=classweights)
        l=loss(output_ts_all, y_ts_all)
        print('test loss: ' + str(l)+ '\n')
        
        thresh=0.5
        cmat=confusion_matrix(y_ts_all, np.array([output_ts_all[i]>(thresh) for i in range (output_ts_all.shape[0])]))
        print(sum(y_ts_all))
        print(cmat)
        N=np.sum(cmat)
            
        
        tp=cmat[1][1]
        fn=cmat[1][0]
        fp=cmat[0][1]
        tn=cmat[0][0]
                
        recall=round(float(tp)/float(tp+fn+1e-6), 3)
        precision=round(float(tp)/float(tp+fp+1e-6), 3)
        accuracy=round(float(tp+tn)/float(N), 3)
        far=round(float(fp)/float(fp+tn))
        #bacc=round(0.5*(float(tp)/float(tp+fn)+float(tn)/float(tn[p]+fp[p])), 3)
        hss=round(2*float(tp*tn-fp*fn)/float((tp+fn)*(fn+tn)+(tp+fp)*(fp+tn)), 3)
        tss=round(float(tp)/float(tp+fn+1e-6)-float(fp)/float(fp+tn+1e-6), 3)
                
        if tss > max_tss:
            max_tss=tss
            max_far=far
            max_hss=hss
            max_recall=recall
            max_precision=precision
            max_acc=accuracy
                
        recall_list.append(recall)
        precision_list.append(precision)
        acc_list.append(accuracy)
        far_list.append(far)
        tss_list.append(tss)
        hss_list.append(hss)
            
        
                    
                    
                        
    

--------- 0 iteration-------------
tensor(3.6415)
loss 1.0830
loss 1.0753
loss 1.0487
loss 1.0182
loss 0.9643
loss 0.9400
loss 0.9226
loss 0.9063
loss 0.8766
tensor(16418.)
tensor([0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0.])
tensor([-0.6732, -0.6259, -0.8795, -0.7864, -0.7312,  1.6167, -0.9842, -0.8629,
        -0.6956,  1.3944, -0.9500, -0.7662, -0.5690,  1.6396,  1.1774, -0.4105,
        -0.4414, -0.5335,  1.5139, -0.9727])
cmat_train: 
[[51187  8508]
 [ 6991  9427]]
loss 0.8700
test loss: tensor(0.7915, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

tensor(7833.)
[[29444  2935]
 [ 3352  4481]]
--------- 1 iteration-------------
tensor(3.6189)
loss 1.0508
loss 1.0187
loss 0.9760
loss 0.9483
loss 0.9392
loss 0.9092
loss 0.8899
loss 0.8876
loss 0.8641
tensor(16455.)
tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1.])
tensor([-0.6215, -1.2581, -0.9313,  0.7132, -1.0691,  1.0466, -1.2092, -1.0697,

In [15]:
avg_recall=round(np.mean(np.array(recall_list)), 3)
std_recall=round(np.std(np.array(recall_list)), 3)
    
avg_precision=round(np.mean(np.array(precision_list)), 3)
std_precision=round(np.std(np.array(precision_list)), 3)
    
avg_acc=round(np.mean(np.array(acc_list)), 3)
std_acc=round(np.std(np.array(acc_list)), 3)
    
avg_far=round(np.mean(np.array(far_list)), 3)
std_far=round(np.std(np.array(far_list)), 3)
max_far=round(np.max(np.array(far_list)), 3)
min_far=round(np.min(np.array(far_list)), 3)
    
avg_hss=round(np.mean(np.array(hss_list)), 3)
std_hss=round(np.std(np.array(hss_list)), 3)
max_hss=round(np.max(np.array(hss_list)), 3)
min_hss=round(np.min(np.array(hss_list)), 3)
    
avg_tss=round(np.mean(np.array(tss_list)), 3)
std_tss=round(np.std(np.array(tss_list)), 3)
max_tss=round(np.max(np.array(tss_list)), 3)
min_tss=round(np.min(np.array(tss_list)), 3)
    

    

In [20]:
print ('threshold: ' +str(0.5)+ '\n')

print('avg recall: '+str(avg_recall) +
      ' (' + str(std_recall) + ')\n')
print('avg precision: ' + str(avg_precision)
      +' (' + str(std_precision) +')\n')
print('avg acc: ' + str(avg_acc) 
      + ' (' + str(std_acc) + ')\n')
print('avg hss: ' + str(avg_hss) 
      + ' (' +str(std_hss) +')\n')
print('avg tss: ' + str(avg_tss)+' ('
      +str(std_tss)+ ')\n')



threshold: 0.5

avg recall: 0.597 (0.01)

avg precision: 0.584 (0.014)

avg acc: 0.838 (0.005)

avg hss: 0.489 (0.009)

avg tss: 0.494 (0.007)

